# Handling a streaming response from iSamples

This is an example using Javascript to handle a streaming JSON response from iSamples uinsg [JSONStrum](https://github.com/xtao-org/jsonstrum).

A request is made for 500,000 records, returning identifier, x, y, z, H3-10 (the [H3](https://h3geo.org/) value at resolution 10)

It takes about 20 seconds to stream all the content. The last record is an "end of response" indicator, hence the total record count is 500,001.

In [1]:
import {JsonStrum} from 'npm:@xtao-org/jsonstrum'

In [2]:
class ISamplesClient {
    constructor(service_url) {
        this.service_url = service_url;
    }

    async getStream(params) {
        const headers = new Headers();
        headers.append("Origin", "localhost")
        const url = new URL(this.service_url + "thing/stream");
        for (const [k, v] of Object.entries(params)) {
            url.searchParams.append(k, v);
        }
        const records = [];
        const strum = JsonStrum({
            // A record. This is where additional record handling would be implemented.
            object: (object) => records.push(object),
            //array: (array) => console.log('array', array),
            // will only parse and emit objects at this level of nesting 
            level: 3,
        });
        
        const decoder = new TextDecoder();
        const response = await fetch(url, {"headers":headers});
        console.log(response.status);
        for await (const chunk of response.body) {
            strum.chunk(decoder.decode(chunk));
        }
        return records;
    }
}

const n_records = 500000;
const params = new URLSearchParams({
    "rows": n_records,
    "fl": "id,x:producedBy_samplingSite_location_longitude,y:producedBy_samplingSite_location_latitude,z:producedBy_samplingSite_location_elevationInMeters,h310:producedBy_samplingSite_location_h3_12",
    "fq": "*:*"
});


let cli = new ISamplesClient("http://central.isample.xyz/isamples_central/");
let records = await cli.getStream(params);
console.log(records.length);


200
500001
